In [28]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import cv2
import time

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

from IPython import display
import csv

# splitting test data and validation data
all_data = np.loadtxt("COMP90086_2023_TLLdataset/train.csv",
                 delimiter=",", dtype=str)

left_imgs = all_data[:,0][1:]
right_imgs = all_data[:,1][1:]

## path for training data
data_dir = "COMP90086_2023_TLLdataset/train"

##split data into 20% testing and 60% training and 20% validation
left_train, left_test, right_train, right_test = train_test_split(left_imgs, right_imgs, random_state=104, test_size = 0.2, shuffle=True)
left_train, left_val, right_train, right_val = train_test_split(left_train, right_train, random_state=104, test_size = 0.25, shuffle=True)
print(right_train)

['dlg' 'eix' 'zqw' ... 'zwb' 'ojs' 'ijp']


In [17]:
##convert image names to gray scale image arrays:
def convert_left(array, data_dir):
    image_str = data_dir+"/left/"+array[0]+".jpg"
    img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
    ##img_array = cv2.resize(img_array, (40, 49))
    total = np.array([img_array])
    
    for val in array[1:]:
        image_str = data_dir+"/left/"+val+".jpg"
        img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
        ##img_array = cv2.resize(img_array, (40, 49))
        total = np.append(total, np.array([img_array]), axis = 0)
    return total

def convert_right(array, data_dir):
    image_str = data_dir+"/right/"+array[0]+".jpg"
    img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
    ##img_array = cv2.resize(img_array, (40, 49))
    total = np.array([img_array])
    
    for val in array[1:]:
        image_str = data_dir+"/right/"+val+".jpg"
        img_array = cv2.imread(image_str, cv2.IMREAD_GRAYSCALE)
        ##img_array = cv2.resize(img_array, (40, 49))
        total = np.append(total, np.array([img_array]), axis = 0)
    return total

In [18]:
left_train_img_arrays = convert_left(left_train, data_dir)
left_val_img_arrays = convert_left(left_val, data_dir)

right_train_img_arrays = convert_right(right_train, data_dir)
right_val_img_arrays = convert_right(right_val, data_dir)
print(left_val_img_arrays.shape)


(400, 245, 200)


In [19]:
# Normalize and reshape the data
left_train_img_arrays = left_train_img_arrays.astype('float32') / 255.
left_val_img_arrays = left_val_img_arrays.astype('float32') / 255.

right_train_img_arrays = right_train_img_arrays.astype('float32') / 255.
right_val_img_arrays = right_val_img_arrays.astype('float32') / 255.


#x_left_train = np.reshape(left_train_img_arrays, (len(left_train_img_arrays), 32, 32, 1))
# x_test = np.reshape(test_data, (len(test_data), 28, 28, 1))

x_left_train = left_train_img_arrays.reshape((len(left_train_img_arrays), np.prod(left_train_img_arrays.shape[1:])))
x_left_val = left_val_img_arrays.reshape((len(left_val_img_arrays), np.prod(left_val_img_arrays.shape[1:])))

x_right_train = right_train_img_arrays.reshape((len(right_train_img_arrays), np.prod(right_train_img_arrays.shape[1:])))
x_right_val = right_val_img_arrays.reshape((len(right_val_img_arrays), np.prod(right_val_img_arrays.shape[1:])))

print("train_images shape:", x_left_train.shape)


train_images shape: (1200, 49000)


In [34]:
# This is the size of our encoded representations
latent_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
# This is our input image
input_img = keras.Input(shape=(49000,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(latent_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(49000, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(latent_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_left_train, x_right_train,
                epochs=35,
                batch_size=256,
                shuffle=True,
                validation_data=(x_left_val, x_right_val))


Epoch 1/35
5/5 [==============================] - 5s 543ms/step - loss: 0.6931 - val_loss: 0.6923
Epoch 2/35
5/5 [==============================] - 2s 396ms/step - loss: 0.6892 - val_loss: 0.6855
Epoch 3/35
5/5 [==============================] - 2s 376ms/step - loss: 0.6804 - val_loss: 0.6790
Epoch 4/35
5/5 [==============================] - 2s 374ms/step - loss: 0.6760 - val_loss: 0.6767
Epoch 5/35
5/5 [==============================] - 2s 385ms/step - loss: 0.6745 - val_loss: 0.6757
Epoch 6/35
5/5 [==============================] - 2s 379ms/step - loss: 0.6731 - val_loss: 0.6738
Epoch 7/35
5/5 [==============================] - 2s 409ms/step - loss: 0.6714 - val_loss: 0.6742
Epoch 8/35
5/5 [==============================] - 2s 382ms/step - loss: 0.6708 - val_loss: 0.6724
Epoch 9/35
5/5 [==============================] - 2s 376ms/step - loss: 0.6700 - val_loss: 0.6722
Epoch 10/35
5/5 [==============================] - 2s 401ms/step - loss: 0.6694 - val_loss: 0.6718
Epoch 11/35
5/5 [==

In [35]:
encoder = keras.Model(inputs=autoencoder.input, outputs=autoencoder.layers[1].output)

In [42]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load and preprocess two images
img_path1 = np.array(['aif'])
img1 = convert_left(img_path1, data_dir)
left_img = img1.astype('float32') / 255.
x_left = left_img.reshape((len(left_img), np.prod(left_img.shape[1:])))
encoded_features_left = encoder.predict(x_left)

right_imgs = np.array(['osr','xjj', 'lyf' , 'mqw', 'dlg' ,'eix' ,'zqw', 'zwb' ,'ojs' ,'ijp'])

for val in right_imgs:
    img_path2 = np.array([val])
    img2 = convert_right(img_path2, data_dir)
    right_img = img2.astype('float32') / 255.
    x_right = right_img.reshape((len(right_img), np.prod(right_img.shape[1:])))

    # use the encoder to get the feature vectors
    encoded_features_left = encoder.predict(x_left)
    encoded_features_right = encoder.predict(x_right)

    # Calculate Euclidean distance
    euclidean_distance = np.linalg.norm(encoded_features_left - encoded_features_right)
    
    print("%s : %d", val, euclidean_distance)

1/1 [==============================] - 0s 34ms/step
%s : %d osr 39.406254
1/1 [==============================] - 0s 24ms/step
%s : %d xjj 23.985691
1/1 [==============================] - 0s 30ms/step
%s : %d lyf 55.649857
1/1 [==============================] - 0s 27ms/step
%s : %d mqw 17.280386
1/1 [==============================] - 0s 35ms/step
%s : %d dlg 5.51256
1/1 [==============================] - 0s 29ms/step
%s : %d eix 9.165424
1/1 [==============================] - 0s 23ms/step
%s : %d zqw 1.4398232
1/1 [==============================] - 0s 25ms/step
%s : %d zwb 3.2566228
1/1 [==============================] - 0s 27ms/step
%s : %d ojs 13.368179
1/1 [==============================] - 0s 28ms/step
%s : %d ijp 17.384405
